This notebook prepare the catalogues that will be analysed by CIGALE for SED fitting and physical parameter estimation.

In [1]:
import numpy as np
import os

from astropy.table import Table

from herschelhelp.filters import correct_galactic_extinction
from herschelhelp.external import convert_table_for_cigale

In [2]:
master_catalogue = Table.read("../../dmu32/dmu32_ELAIS-N1/data/ELAIS-N1_20170816_cigale.fits")

In [3]:
len(master_catalogue)

4026292

# Best sources

Define a good far-IR measurement as:
- an existing flux in the band;
- the flag from XID+ must not be set;
- the signal to noise ratio must be over 2.

In [4]:
good = {}
for band in ['pacs_green', 'pacs_red', 'spire_250', 'spire_350', 'spire_500']:
    good[band] = (~np.isnan(master_catalogue['f_{}'.format(band)]) & 
                                   ~master_catalogue['flag_{}'.format(band)])
    good[band][good[band]] &= (master_catalogue[good[band]]['f_{}'.format(band)] /
                              master_catalogue[good[band]]['ferr_{}'.format(band)] >= 2)

In [5]:
combined_good = np.sum(list(good.values()), axis=0) >= 2

In [6]:
print("Number of good sources: {}".format(np.sum(combined_good)))

Number of good sources: 51153


# Main catalogue for CIGALE

In [7]:
best_catalogue = master_catalogue[combined_good].copy()

In [8]:
# Correction for galactic extinction
best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [9]:
# Convertion to CIGALE format
os.environ['DEBUG'] = 'info'
best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True)

/opt/anaconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/numpy/ma/core.py:852: RuntimeWarning: invalid value encountered in greater_equal
  return umath.absolute(a) * self.tolerance >= umath.absolute(b)
/opt/anaconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/table/column.py:1096: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)
/opt/herschelhelp_python/herschelhelp/external.py:120: RuntimeWarning: invalid value encountered in greater
  below_ly = np.where(FILTER_MIN_LAMBDAS[band] < lyman_limit_at_z)[0]
INFO:herschelhelp.external:For 2572 sources, the band wfc_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1745 sources, th

## Band selection

We want to use only one filter for similar bands. We define an order of preference and set to NaN the flux in the lower prefered bands when a prefered band is available. Some band may have a 0 flux, we set there values to NaN.

In [10]:
g_bands = ['suprime_g', 'megacam_g', 'wfc_g', 'gpc1_g']
u_bands = ['megacam_u', 'wfc_u']
r_bands = ['suprime_r', 'megacam_r', 'wfc_r', 'gpc1_r']
i_bands = ['suprime_i', 'wfc_i']
z_bands = ['suprime_z', 'megacam_z', 'wfc_z']
y_bands = ['suprime_y', 'gpc1_y']

def remove_unneeded_fluxes(list_of_bands):
    for band_idx, band in enumerate(list_of_bands[:-1]):
        mask = ~np.isnan(best_catalogue[band])
        for lower_band in list_of_bands[band_idx+1:]:
            best_catalogue[lower_band][mask] = np.nan
            best_catalogue["{}_err".format(lower_band)][mask] = np.nan

In [11]:
remove_unneeded_fluxes(g_bands)
remove_unneeded_fluxes(u_bands)
remove_unneeded_fluxes(r_bands)
remove_unneeded_fluxes(i_bands)
remove_unneeded_fluxes(z_bands)
remove_unneeded_fluxes(y_bands)

/opt/anaconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/table/column.py:1096: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)


In [12]:
best_catalogue.write("data_tmp/ELAIS-N1_cigale_best_extcor_20171019.fits")

# Catalogue using spectroscopic redshift

In [13]:
best_catalogue = master_catalogue[combined_good].copy()

In [14]:
best_catalogue.remove_column("redshift")
best_catalogue["zspec"].name = "redshift"

In [15]:
best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]

In [16]:
print("Number of sources with z-spec: {}".format(len(best_catalogue)))

Number of sources with z-spec: 1532


In [17]:
# Correction for galactic extinction
best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [18]:
# Convertion to CIGALE format
os.environ['DEBUG'] = 'info'
best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True)

/opt/anaconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/numpy/ma/core.py:852: RuntimeWarning: invalid value encountered in greater_equal
  return umath.absolute(a) * self.tolerance >= umath.absolute(b)
/opt/anaconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/table/column.py:1096: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)
INFO:herschelhelp.external:For 70 sources, the band wfc_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 46 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 26 

In [19]:
remove_unneeded_fluxes(g_bands)
remove_unneeded_fluxes(u_bands)
remove_unneeded_fluxes(r_bands)
remove_unneeded_fluxes(i_bands)
remove_unneeded_fluxes(z_bands)
remove_unneeded_fluxes(y_bands)

/opt/anaconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/table/column.py:1096: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)


In [20]:
best_catalogue.write("data_tmp/ELAIS-N1_cigale_best_extcor_zspec_20171019.fits")